In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import cartopy 
import numpy as np
import xarray as xr
import math as m
from datetime import datetime
from datetime import timedelta
from glob import glob
from parcels import (grid, Field, FieldSet, SummedField, VectorField, ParticleSet, JITParticle, ScipyParticle, AdvectionRK4, ErrorCode, ParticleFile, Variable, plotTrajectoriesFile)



path_dataSMOC = '/data/oceanparcels/input_data/CMEMS/GLOBAL_ANALYSIS_FORECAST_PHY_001_024_SMOC/'

#### FIELDSET SMOC #####
def choosefield(whichfield):
    if whichfield=='hourly':
        SMOCfiles = path_dataSMOC+'SMOC_202003*.nc'
    elif whichfield== 'mean':
        SMOCfiles = '/scratch/tycho/SMOC*2020.nc'
    else:
        raise ValueError('wrong field')
    filenames_SMOC = {'U': SMOCfiles,
                       'V': SMOCfiles}
    variables_SMOC = {'U' : 'uo',
                 'V' : 'vo'}
    dimensions_SMOC = {'U': {'time' : 'time',
                        'lat' : 'latitude',
                        'lon' : 'longitude',
                       'depth': 'depth'},
                  'V': {'time' : 'time',
                        'lat' : 'latitude',
                        'lon' : 'longitude',
                       'depth': 'depth'}}
    indices_SMOC = {'lon' : range(1380, 1620),
               'lat' : range(1620,1740)}
    #mesh = fieldFiles[0]
    filenames_SMOC = {'U': SMOCfiles,
                       'V': SMOCfiles}
    variables_tides = {'U' : 'utide',
                 'V' : 'vtide'}
    dimensions_tides = {'U': {'time' : 'time',
                        'lat' : 'latitude',
                        'lon' : 'longitude',
                       'depth': 'depth'},
                  'V': {'time' : 'time',
                        'lat' : 'latitude',
                        'lon' : 'longitude',
                       'depth': 'depth'}}
    indices_SMOC = {'lon' : range(1380, 1620),
               'lat' : range(1620,1740)}
    if whichfield=='hourly':
        fieldset_ns = FieldSet.from_netcdf(filenames_SMOC, 
                                    variables_SMOC, 
                                    dimensions_SMOC,
                                    indices_SMOC,
                                    allow_time_extrapolation = True,
                                   )
        fieldset_tides = FieldSet.from_netcdf(filenames_SMOC, 
                            variables_tides, 
                            dimensions_tides,
                            indices_SMOC,
                            allow_time_extrapolation = True,
                           )
    elif whichfield=='mean':
        fieldset_ns = FieldSet.from_netcdf(filenames_SMOC, 
                                    variables_SMOC, 
                                    dimensions_SMOC,
                                    allow_time_extrapolation = True,
                                   )
        fieldset_tides = FieldSet.from_netcdf(filenames_SMOC, 
                            variables_tides, 
                            dimensions_tides,
                            allow_time_extrapolation = True,
                           )
    fieldset= FieldSet(U=fieldset_ns.U+fieldset_tides.U, V=fieldset_ns.V+fieldset_tides.V)
    return fieldset


def aging(particle, fieldset,time):
    particle.Age+=particle.dt
    if particle.Age>3600*24*15:
        particle.delete()


def deleteparticle(particle,fieldset,time):
    """ This function deletes particles as they exit the domain and prints a message about their attributes at that moment
    """
    particle.delete()
    
    
def SampleVel(particle, fieldset, time):
    particle.u = fieldset.U[time, particle.depth, particle.lat, particle.lon]
    particle.v = fieldset.V[time, particle.depth, particle.lat, particle.lon]
            

 


##### PARTICLE SET ####
npar = 10
class BuoyParticle(JITParticle):
    Age=Variable('Age',initial=0)
    u = Variable('u', initial=0)
    v = Variable('v', initial=0)
    
lon = np.linspace(-65,-45,npar)
lat=np.linspace(55,65,npar)
LON, LAT = np.meshgrid(lon,lat)
times = np.full(np.shape(LON),np.datetime64('2020-03-02'))
depths = np.full_like(LON, 0.494025)




In [ ]:
##SMOC
fieldset=choosefield('mean')

pset = ParticleSet.from_list(fieldset=fieldset,   # the fields on which the particles are advected
                             pclass=BuoyParticle,  # the type of particles (JITParticle or ScipyParticle)
                             lon=LON, # a vector of release longitudes 
                             lat=LAT,    # a vector of release latitudes
                              time=times,
                            depth=depths,
                            repeatdt = )


#Execute  the kernel and write to file
output_file = pset.ParticleFile(name="/scratch/tycho/diff_model_WT_mean.nc") # the file name and the time step of the outputs
pset.execute(AdvectionRK4+age_kernel+sample_kernel,                 # the kernel (which defines how particles move)
             runtime=3600*24*16,    # the total length of the run
             dt=600,      # the timestep of the kernel
             recovery= {ErrorCode.ErrorOutOfBounds:deleteparticle}, #delete particle when it gets out of the domain
             output_file=output_file)
output_file.close()


